Cristescu Adrian Iulian

# Logistic Regression of a portfolio.
________________
### The objective of this exercise is to find, with logistic regression, which on the regressors impact the most the Probability of Default (PD) after 90 days.
______
Dataset: 3.364 rows of full data (no blank values)
The dataset contains the information of around 3300 individuals and the exercise is trying to predict the default or non default of the variable "BAD" which indicates 0 for NON-Default and 1 for Default.
____________
The considered variables on the dataset are:

- BAD: 1 = applicant defaulted on loan or seriously delinquent; 0 = applicant paid loan
- LOAN: Amount of the loan request
- MORTDUE: Amount due on existing mortgage
- VALUE: Value of current property
- REASON: DebtCon = debt consolidation; HomeImp = home improvement
- JOB: Occupational categories
- YOJ: Years at present job
- DEROG: Number of major derogatory reports
- DELINQ: Number of delinquent credit lines
- CLAGE: Age of oldest credit line in months
- NINQ: Number of recent credit inquiries
- CLNO: Number of credit lines
- DEBTINC: Debt-to-income ratio

H. Scheule, D. Roesch, B. Baesens, Credit Risk Analytics: The R Companion, Scheule Roesch Baesens, 2017.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import numpy as np
from matplotlib.ticker import FuncFormatter
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, roc_curve,classification_report
import warnings
import xgboost as xgb
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from scipy.stats import zscore
from imblearn.over_sampling import SMOTE , BorderlineSMOTE, ADASYN
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.svm import SVC
from itertools import combinations

__________________
### Let's see how many rows we have and also drop the rows with empty values

In [ ]:
%%time
data = pd.read_csv('hmeq.csv')
# Separate rows with 0 and 1 values in the 'BAD' column
#zeros = databeforecut[databeforecut['BAD'] == 0]
#ones = databeforecut[databeforecut['BAD'] == 1]

# Randomly sample 10% of the rows with 0 values
#sampled_zeros = zeros.sample(frac=0.32, random_state=0)


data.dropna(inplace=True)
data.info()

______________
### Define the categorical values: Job profession and Reason of loan

In [3]:
# Define manual mapping dictionaries for 'REASON' and 'JOB'
reason_mapping_manual = {'DebtCon': 0, 'HomeImp': 1}  # Assigning numbers starting from 0
job_mapping_manual = {'Other': 0, 'Office': 1, 'Mgr': 2, 'ProfExe': 3, 'Sales': 4, 'Self': 5}  # Assigning numbers starting from 0

# Apply manual mapping for 'REASON' and 'JOB'
data['REASON'] = data['REASON'].map(reason_mapping_manual)
data['JOB'] = data['JOB'].map(job_mapping_manual)

In [ ]:
pd.options.display.float_format = '{:.2f}'.format  # Format float values to 2 decimals
pd.set_option('display.max_columns', None)  # Show all columns without truncating
pd.set_option('display.width', 1000)  # Control width to avoid line breaks

# Show the descriptive statistics
stats = data.describe()
print(stats)

______________
### Correlation Matrix

In [ ]:
m_data=data.corr()
m_data.style.background_gradient(cmap='viridis', axis=None, low=0, high=1, vmin=-1, vmax=1, subset=None).format("{:.2f}")

### Let's see the difference of Defaults and Non Defaults

In [6]:
#remove the variable we want to regress against, which is default after 90 days
X = data.drop('BAD', axis=1) #we remove the "default>90days" variable and we plot it as dependent variable below
y = data['BAD']

In [ ]:
filtered_data = data[data['BAD'].isin([0, 1])]
value_counts = filtered_data['BAD'].value_counts()
colors = ['blue' if x == 0 else 'red' for x in value_counts.index]
plt.bar(value_counts.index, value_counts.values, color=colors)
for i, v in enumerate(value_counts.values):
    plt.text(i, v + 10, str(v), color='black', ha='center')
plt.xlabel('BAD')
plt.ylabel('Count')
plt.title('Counts of BAD Values')
plt.xticks(value_counts.index)
plt.show()

-------------------
### We start training the model. We have:
- **Training Set**: Used to train the machine learning model by providing labeled data for learning patterns and relationships between features and target outputs.
- **Validation Set**: Used to assess and tune the model during training, ensuring it generalizes well to unseen data by evaluating its performance on data not used in training.
- **Test Set**: Used to provide an unbiased evaluation of the final trained model's performance on completely unseen data, measuring how well it generalizes to new observations.


In [8]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)  # 30% test set
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42, stratify=y_temp)  # 33% val, 30% test

- In the first sentence we say that 70% of the dataset is used for training and the rest of 30% is put aside
- in the second sentence we further split the temporary set into a validation and test set, amounting at 33%.
- The stratification parameter ensures that the class distribution in y is preserved in both splits, keeping representative samples across all sets. This ensures that since Defaults and Non-Defaults aren't uniform (different number of observations). By using stratification we enture that the proportion of observations are mantained during the training. In our case, the ratio is almost 10 to 1, hence, during the training, the sets keep this ratio.
- ---------------------
### SMOTE (Synthetic Minority Over-sampling Technique)
    - We generate synthetic examples of the minority class by interpolating between existing minority class instances. This technique helps balance class distribution in imbalanced datasets (ours) by enhancing the model's ability to learn from minority class examples without introducing biaqs frim simply duplication data points.
    - The synthetic examples are generated by selecting the minority class (Default) instance at random and computing the k-nearest neighbors for this instance. The examples are then created along the line segments joining these neighbors. 

## Step 1:
### SMOTE - BORDERLINE
    - We use a particular type of SMOTE: the Borderline version. This type manages better the wrong classification problem of the minority class which is located near the border between classes. These cohorts are difficult to classify, hence, they have higher probability of being classified wrongly by models. Borderline SMOTE focuses on the generation of synthetic cohorts near the borderline of minority/majority classes. It adresses the istances that are more difficult to classify. 
    
## Step 2:
### Data Normalization
    - StandardScaler is used to ensure all features are on the same scale, aiding machine learning models to their performance and convergence.

## Step 3:
### Transform back to DataFrame
    - We now transform back the normalized data into a DataFrame


In [9]:
#Step 1
#Apply SMOTE (Synthetic minority oversampling technique)
smote=BorderlineSMOTE(sampling_strategy='minority', kind='borderline-1')
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

#Step 2
#normalize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_sm)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

#Step 3
# Convert the scaled data back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_scaled_df = pd.DataFrame(X_val_scaled, columns=X_val.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

______________________
# MODELS - Introduction of used terms and metrics
- **ROC Curve** : Receiver Operating Characteristic Curve
  - It is a graphical representation of the performance of a binary classifier system as its discrimination threshold is varied.
  - It plots the True Positive Rate against False Positive Rate ar various threshold settings.
  - A diagonal ROC represents random guessing, while an ideal classifier will have its ROC curve reaching the top-left corner of the plot.
- **AUC** : Area under the ROC Curve
  - It quantifies the overall performance of a binary classification model based on its ability to discriminate between positive and negative examples.
  - AUC ranges from 0 to 1, where an AUC of 0.5 indicates a model that performs no better than random, and an AUC of 1 indicates a perfect classifier.
  - A higher AUC value generally indicates a better-performing model in terms of its ability to distinguish between positive and negative classes.
- **Validation Set** : it is used during model training to tune hyperparameters. Different combinations of parameters are insertet manually until the AUC for the Validation Set is high. After the best parameters are found, they are inserted as unique parameters for the final test set.
- **Test Set** : This set is used only once after the parameters has been chosen.
- **Precision** : The proportion of true positive predictions among all positive predictions made. It indicates how many selected items are relevant:
  - $\frac{True Positives}{True Positives + False Positives}$
- **Accuracy** : The proportion of true positive and true negative predictions among all predictions made. It indicates the overall correctness of the model:
  - $\frac{True Positives + True Negatives}{Total Predictions}$
- **Recall** : The proportion of true positive predictions among all actual positive cases. It indicates how many relevant items are selected.
  - $\frac{True Positives}{True Positives + False Negatives}$
- **F1-Score** : The harmonic mean of precision and recall. It balances the two metrics, providing a single measure of a test's accuracy.
  - $\frac{Precision \times Recall}{Precision + Recall}$
- **CV - Cross Validation** : In every model, there is the "grid_search" code which join the estimators with the chosen model. There is also the metric "cv" which is the cross-validation. For every mode I've chosen 10 cv, which means it divides the set in 10 splits and performs the tests on them, averaging the result in the end

### Predicted Probabilities: How are they computed?
#### Logistic Regression:

After training, when making predictions on new data $( X_{\text{test}} )$, the model computes a linear combination of the input features weighted by learned coefficients. This linear combination is transformed using the logistic (sigmoid) function:

- \[$$ P(y = 1 \mid X) = \frac{1}{1 + e^{-\left(\beta_0 + \beta_1 X_1 + \beta_2 X_2 + \ldots + \beta_n X_n\right)}} $$]

##### Explanation:
- $( \beta_0, \beta_1, \ldots, \beta_n )$ are the coefficients learned during training.
- $( X_1, X_2, \ldots, X_n )$ are the features of the new data point.

##### Output:
- The result of the sigmoid function gives the predicted probability \[$P(y=1 \mid X)$], which represents the model’s confidence that the output $( y )$ (default/non-default, in this case) is 1 (default) given the features $ (X) $]
t.
point.
a point.
 point.


______________________________
#### Decision Tree

A Decision Tree makes predictions by traversing from the root of the tree to a leaf node, making decisions at each node. The predicted probability of a class is the proportion of samples of that class in the leaf node:

\[ $P(y = c \mid X) = \frac{N_c}{N}$ \]

where $N_c$ is the number of samples of class $c$ in the leaf node and $N$ is the total number of samples in the leaf node.

_________________________

#### Random Forest

Random Forest predicts probabilities by averaging the probabilities predicted by each individual tree in the forest. For a binary classification problem:

$$\[ P(y = 1 \mid X) = \frac{1}{T} \sum_{t=1}^{T} P_t(y = 1 \mid X) \]$$

where $T$ is the number of trees in the forest and $P_t(y = 1 \mid X)$ is the probability predicted by the $t$-th tree.

______________________

#### AdaBoost

AdaBoost computes the predicted probabilities by combining the weighted predictions of weak classifiers. The weight of each classifier depends on its accuracy. The final prediction is given by:

$$\[ F(x) = \sum_{m=1}^{M} \alpha_m h_m(x) \]$$

where $M$ is the number of classifiers, $\alpha_m$ is the weight of the $m$-th classifier, and $h_m(x)$ is the prediction of the $m$-th classifier. The predicted probabilities are then given by:

$$\[ P(y = 1 \mid X) = \frac{1}{1 + e^{-F(x)}} \]$$

_____________________________
#### Gradient Boosting

Similarly, predicts probabilities indirectly through a score $F(x)$, which is the cumulative sum of predictions from individual trees, weighted by learning rates $\eta$:

$$\[ F(x) = \sum_{m=1}^{M} \eta \cdot h_m(x) \]$$

where $h_m(x)$ is the prediction from the $m$-th tree. Predicted probabilities are then given by:

$$\[ P(y = 1 \mid X) = \frac{1}{1 + e^{-F(x)}} \]$$

___________________________________
#### XGBoost

XGBoost, like other gradient boosting methods, predicts probabilities by combining the predictions of multiple trees. The model computes a score $F(x)$, which is the sum of the predictions from individual trees weighted by the learning rate $\eta$:

$$\[ F(x) = \sum_{m=1}^{M} \eta \cdot h_m(x) \]$$

where $M$ is the number of trees, $\eta$ is the learning rate, and $h_m(x)$ is the prediction from the $m$-th tree. The predicted probabilities are then given by:

$$\[ P(y = 1 \mid X) = \frac{1}{1 + e^{-F(x)}} \]$$

---------------
# XGBoost model:
 ### XGB is a boosting machine learning model with a great performance. It's the first model considered since it has some PROs:
 - **High Predictive Accuracy**: it has a superior performance in predictive accuracy. It handles complex relationships and dependencies in data through ensemble learning techniques like gradient boosting
 - **Missing Values**: it handles very well the missing values. Reduces the preprocessing effots.
 - **Learning**: After the first predictions, the residuals are calculated and a new model is trained to predict those errors. The new predictions are added to the old model. This process in as many times as indicated. Below in "param_grid_xgb" we have n_estimators=450, which is the number of boosting rounds.
 - **Flexibility**: can handle a variety of data types and can be used for classification and regression.
 - **Scalability**: it can handle large datasets and can train them faster
 - **Regularization**: includes regularization techniques such as L1 and L2 ti prevent overfitting and improve generalization.
      - **L1**: adds the abs. value of the coefficients to the loss function. It prodices sparse models, meaning it drives some coefficients to exactly zero. It is useful for identifying the most important features in the dataset. If $\beta$ are the model coefficients, it adds $\lambda \sum_i |\beta_i|$ to the loss function where $\lambda$ is a regularization parameter that controls the strenght of the penalty
      - **L2**: adds the squared values of the coefficients $\lambda \sum_i \beta_i$ to the loss function. It distributes the penalty more evenly across all coefficients, shrinking the towards zero but not necessarily making theme xactly zero. This helps to keep all features in the model, reducing the risk of overfitting without eliminating any features.
 - **Feature Importance**: It helps to identify which features have the most impact on predictions and understand the underlying relationships in data


In [ ]:
%%time
#XGBoost model
param_grid_xgb = { #the parameters were chosen manually after repeated attemps with different values. These gave the best AUC
    'learning_rate': [0.04, 0.05, 0.03, 0.09], #values tipically between 0.01 and 0.3: For lower learning rate it provides more accuracy and less speed but also 
                            # can produce overfitting. The opposite happens with higher learning rate.
    'n_estimators': [500, 400, 450], #number of boosting round
    'max_depth':[14, 10, 12] #is the maximum depth of a tree. It controls the complexity of the model. Bigger the values the more complex patters
                    #but it can lead to overfitting
}

# Initialize the XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Set up the grid search with cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, scoring='accuracy', cv=10, n_jobs=-1)

# Fit the GridSearchCV to the training data
grid_search.fit(X_train_scaled_df, y_train)

# Get the best parameters from the grid search
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# Get the best estimator
best_xgb_model = grid_search.best_estimator_
# Validate the model on the validation set
y_val_pred_xgb = best_xgb_model.predict(X_val_scaled_df)
y_val_pred_proba_xgb = best_xgb_model.predict_proba(X_val_scaled_df)[:, 1]

# Print classification report for validation set
print("Validation Set Classification Report")
print(classification_report(y_val, y_val_pred_xgb))

# Print AUC score for validation set
print(f"AUC (Validation): {roc_auc_score(y_val, y_val_pred_proba_xgb):.4f}")

In [ ]:
%%time
# Initialize the XGBoost model with the best parameters extracted from GridSearchCV
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='auc', random_state=42, **best_params_xgb)

# Train the model on the training data
xgb_model.fit(X_train_scaled_df, y_train_sm)

# Evaluate the model on the test data
y_test_pred_xgb = xgb_model.predict(X_test_scaled_df)
y_test_pred_proba_xgb = xgb_model.predict_proba(X_test_scaled_df)[:, 1]

# Print classification report for test set
print("Test Set Classification Report")
print(classification_report(y_test, y_test_pred_xgb))

# Print AUC score for test set
print(f"AUC (Test): {roc_auc_score(y_test, y_test_pred_proba_xgb):.4f}")

In [ ]:
# Calculate ROC curve for test set
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba_xgb)

# Calculate AUC score
auc_score = roc_auc_score(y_test, y_test_pred_proba_xgb)

# Feature importances for XGBoost
xgb_importances = best_xgb_model.feature_importances_
xgb_df = pd.DataFrame(data=xgb_importances, index=X_train_scaled_df.columns, columns=['Value']).sort_values('Value', ascending=False)

# Create a 1x2 subplot layout
fig, axs = plt.subplots(1, 2, figsize=(16, 5))

# Plot ROC curve
axs[0].plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {auc_score:.4f})')
axs[0].plot([0, 1], [0, 1], color='gray', linestyle='--')  # Plotting the diagonal line for random model
axs[0].set_xlim([0.0, 1.0])
axs[0].set_ylim([0.0, 1.05])
axs[0].set_xlabel('False Positive Rate (FPR)')
axs[0].set_ylabel('True Positive Rate (TPR)')
axs[0].set_title('Receiver Operating Characteristic (ROC) Curve')
axs[0].legend(loc='lower right')

# Plot feature importances
axs[1].barh(xgb_df.index, xgb_df['Value'], color='blue')
axs[1].set_xlabel('Feature Importance')
axs[1].set_ylabel('Feature Name')
axs[1].set_title('XGBoost Feature Importances')

plt.tight_layout()
plt.show()

In [ ]:
# Function to plot impact of top features on default probability
def plot_feature_impact(top_features, model_name):
    plt.figure(figsize=(15, 3))  # Adjust figsize as needed
    num_features = len(top_features)
    num_cols = 3  # Number of columns for subplots

    # Calculate number of rows needed
    num_rows = (num_features // num_cols) + (num_features % num_cols > 0)

    for i, feature in enumerate(top_features):
        plt.subplot(num_rows, num_cols, i + 1)
        sns.regplot(x=X[feature], y=y, logistic=True, ci=None, scatter_kws={"s": 10, "alpha": 0.5})
        plt.title(f'Impact of {feature} on Default ({model_name})')
        plt.xlabel(feature)
        plt.ylabel('Probability of Default')
        plt.tight_layout()

    plt.tight_layout()
    plt.show()

# Example usage for XGradient Boosting (GB) model
top_features_xgb = xgb_df.head(3).index  # Select top 3 features for Gradient Boosting
plot_feature_impact(top_features_xgb, 'Extreme Gradient Boosting')

In [ ]:
# Compute confusion matrix for test set
cm = confusion_matrix(y_test, y_test_pred_xgb)

# Create ConfusionMatrixDisplay object
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Non-Default', 'Default'])

# Plot confusion matrix with annotations
fig, ax = plt.subplots(figsize=(6, 4))
disp.plot(cmap='Blues', ax=ax, values_format='.4g')

plt.title('Confusion Matrix - XGBoost Classifier (Test Set)')
plt.tight_layout()
plt.show()

# Random Forest Classifier model:
 ### It is an ensemble learning method that constructs a multitude of decision trees during training.
 - **Bootstrap Sampling**: Each tree in the Random Forest is trained on a bootstrap sample of the data, which involves randomly selecting samples with replacement from the training set.
 - **Feature Randomness**: At each split in the decision tree, a random subset of features is considered. This randomness helps to decorrelate the trees and reduce overfitting.
 - **Voting Mechanism**: For classification tasks, the final prediction of the Random Forest is determined by a majority vote (mode) of the predictions of its constituent trees.
 - **Handling of Missing Values**: It handles missing values in the dataset by using surrogate splits in the trees, reducing the need for imputation or preprocessing.
 - **Scalability**: It is scalable and can efficiently handle large datasets and high-dimensional feature spaces. The training process can be parallelized across multiple processors.
 - **Robust to Overfitting**: By averaging predictions across multiple trees (bagging) and introducing randomness in feature selection and tree construction, Random Forest tends to generalize well and is less prone to overfitting compared to individual decision trees.
 - **Easy to Tune**: Although it has several hyperparameters (e.g., number of trees, maximum depth of trees), it is relatively easier to tune compared to some other complex models like neural networks.
 - **Handling Imbalanced Data**: It handles imbalanced datasets by balancing class weights or using techniques like SMOTE (Synthetic Minority Over-sampling Technique) during training, which is beneficial for classification tasks with unequal class distributions.

#### Validation set

In [ ]:
%%time
#Random Forest model
param_grid_rf = {
    'n_estimators': [385, 380], #Higher values can improve performance at the cost of increased computational 
                                #resources and potential overfitting. Here, we are testing 370 and 380 trees.
                                #Controls the number of decision trees in the forest.
    
    'max_depth': [None, 5, 1],#A deeper tree can model more complex relationships but increases the risk of overfitting
    
    'min_samples_split': [2],#Higher values prevent the model from learning overly specific patterns, potentially reducing overfitting.
    
    'min_samples_leaf': [1 ],# Similar to min_samples_split, higher values can prevent the model from becoming too specific. 
    'criterion': ['gini'],  # Criterion for splitting (you can test both)
    'max_features': ['sqrt', None]  # Number of features to consider at each split
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Set up the grid search with cross-validation
grid_search = GridSearchCV(estimator=rf_model,param_grid=param_grid_rf, scoring='roc_auc', cv=10, n_jobs=-1, verbose=2, error_score='raise')

# Fit the GridSearchCV to the training data
grid_search.fit(X_train_scaled_df, y_train_sm)

# Get the best parameters from the grid search
best_params_rf = grid_search.best_params_
print(f"Best parameters found: {best_params_rf}")

# Get the best estimator
best_rf_model = grid_search.best_estimator_

# Validate the model on the validation set
y_val_pred_rf = best_rf_model.predict(X_val_scaled_df)
y_val_pred_proba_rf = best_rf_model.predict_proba(X_val_scaled_df)[:, 1]

# Print classification report for validation set
print("Validation Set Classification Report")
print(classification_report(y_val, y_val_pred_rf))

# Print AUC score for validation set
print(f"AUC (Validation): {roc_auc_score(y_val, y_val_pred_proba_rf):.4f}")

#### Test set

In [ ]:
%%time
# Initialize the RandomForestClassifier with the best parameters extracted from GridSearchCV
rf_model = RandomForestClassifier(random_state=42, **best_params_rf)

# Train the model on the training data
rf_model.fit(X_train_scaled_df, y_train_sm)

# Evaluate the final model on the test data
y_test_pred_rf = rf_model.predict(X_test_scaled_df)
y_test_pred_proba_rf = rf_model.predict_proba(X_test_scaled_df)[:, 1]

# Print classification report for test set
print("Test Set Classification Report")
print(classification_report(y_test, y_test_pred_rf))

# Print AUC score for test set
print(f"AUC (Test): {roc_auc_score(y_test, y_test_pred_proba_rf):.4f}")

In [ ]:
# Calculate ROC curve for test set
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba_rf)

# Calculate AUC score
auc_score = roc_auc_score(y_test, y_test_pred_proba_rf)

# Feature importances for Random Forest
rf_importances = best_rf_model.feature_importances_
rf_df = pd.DataFrame(data=rf_importances, index=X_train_scaled_df.columns, columns=['Value']).sort_values('Value', ascending=False)

# Create a 1x2 subplot layout
fig, axs = plt.subplots(1, 2, figsize=(16, 5))

# Plot ROC curve
axs[0].plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {auc_score:.4f})')
axs[0].plot([0, 1], [0, 1], color='gray', linestyle='--')  # Plotting the diagonal line for random model
axs[0].set_xlim([0.0, 1.0])
axs[0].set_ylim([0.0, 1.05])
axs[0].set_xlabel('False Positive Rate (FPR)')
axs[0].set_ylabel('True Positive Rate (TPR)')
axs[0].set_title('Receiver Operating Characteristic (ROC) Curve')
axs[0].legend(loc='lower right')

# Plot feature importances
axs[1].barh(rf_df.index, rf_df['Value'], color='green')
axs[1].set_xlabel('Feature Importance')
axs[1].set_ylabel('Feature Name')
axs[1].set_title('Random Forest Feature Importances')

plt.tight_layout()
plt.show()

In [ ]:
# Function to plot impact of top features on default probability
def plot_feature_impact(top_features, model_name):
    plt.figure(figsize=(15, 3))  # Adjust figsize as needed
    num_features = len(top_features)
    num_cols = 3  # Number of columns for subplots

    # Calculate number of rows needed
    num_rows = (num_features // num_cols) + (num_features % num_cols > 0)

    for i, feature in enumerate(top_features):
        plt.subplot(num_rows, num_cols, i + 1)
        sns.regplot(x=X[feature], y=y, logistic=True, ci=None, scatter_kws={"s": 10, "alpha": 0.5})
        plt.title(f'Impact of {feature} on Default ({model_name})')
        plt.xlabel(feature)
        plt.ylabel('Probability of Default')
        plt.tight_layout()

    plt.tight_layout()
    plt.show()

# Example usage for Gradient Boosting (GB) model
top_features_rf = rf_df.head(3).index  # Select top 3 features for Gradient Boosting
plot_feature_impact(top_features_rf, 'Random Forest')

In [ ]:
# Compute confusion matrix for test set
cm = confusion_matrix(y_test, y_test_pred_rf)

# Create ConfusionMatrixDisplay object
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Non-Default', 'Default'])

# Plot confusion matrix with annotations
fig, ax = plt.subplots(figsize=(6,4))
disp.plot(cmap='Blues', ax=ax, values_format='.4g')

plt.title('Confusion Matrix - Random Forest (Test Set)')
plt.tight_layout()
plt.show()

# Logistic Regression Model
### It is a linear model that models the relationship between the dependent binary variable (Default and Non Default) and one or more indipendent variables using a logistic function
 - **Probabilistic Model**: It models the probability that an instance belongs to a particular class (usually the positive class) using the logistic (sigmoid) function, which maps any real-valued input to a value between 0 and 1.
 - **Binary Classification**: Logistic Regression is primarily used for binary classification tasks where the response variable is categorical with two levels (e.g., yes/no, 0/1).
 - **Coefficient Interpretation**: The coefficients (weights) learned by Logistic Regression represent the influence of each feature on the probability of the outcome. Positive coefficients indicate a positive association with the log-odds of the outcome, while negative coefficients indicate a negative association.
 - **Log-Likelihood Optimization**: it maximizes the likelihood function (or equivalently minimizes the negative log-likelihood) to estimate the optimal coefficients. This is typically done using optimization algorithms like Gradient Descent.
 - **Regularization**: Can include regularization techniques like L1 (Lasso) and L2 (Ridge) regularization to penalize large coefficients and prevent overfitting. This is controlled by the regularization parameter (C or λ).
### CONS
- **Feature Scaling**: Logistic Regression is sensitive to the scale of the features. It is common practice to scale features to a similar range (e.g., using StandardScaler) before fitting the model to ensure stable and optimal performance.
- **Assumptions**: Logistic Regression assumes a linear relationship between the independent variables and the log-odds of the dependent variable. It also assumes that there is little or no multicollinearity among the independent variables.

#### Validation set

In [ ]:
%%time

# Define parameter grid for Logistic Regression
param_grid_lr = {
    'C': [0.5, 0.35, 0.6, 0.7, 0.4],  # Regularization parameter, by attempts, 0.7 is the best and 0.1 is the next best one
    'penalty': ['l1', 'l2']# Regularization type, the more efficient is the l1.
}

# Initialize Logistic Regression model
logistic_reg = LogisticRegression(max_iter=5000, solver='liblinear', random_state=42)

# Set up the grid search with cross-validation
grid_search_lr = GridSearchCV(estimator=logistic_reg, param_grid=param_grid_lr, scoring='roc_auc', cv=10, n_jobs=-1)

# Fit the GridSearchCV to the training data
grid_search_lr.fit(X_train_scaled_df, y_train_sm)

# Get the best parameters from the grid search
best_params_lr = grid_search_lr.best_params_
print(f"Best parameters found: {best_params_lr}")

# Get the best estimator
best_lr_model = grid_search_lr.best_estimator_
# Validate the model on the validation set
y_val_pred_lr = best_lr_model.predict(X_val_scaled_df)
y_val_pred_proba_lr = best_lr_model.predict_proba(X_val_scaled_df)[:, 1]

# Print classification report for validation set
print("Logistic Regression Validation Set Classification Report")
print(classification_report(y_val, y_val_pred_lr))

# Print AUC score for validation set
print(f"AUC (Validation): {roc_auc_score(y_val, y_val_pred_proba_lr):.4f}")

#### Test set

In [ ]:
%%time
# Initialize the RandomForestClassifier with the best parameters extracted from GridSearchCV
lr_model = LogisticRegression(max_iter=5000, solver='saga', random_state=42, **best_params_lr)

# Train the model on the training data
lr_model.fit(X_train_scaled_df, y_train_sm)

# Evaluate the final model on the test data
y_test_pred_lr = lr_model.predict(X_test_scaled_df)
y_test_pred_proba_lr = lr_model.predict_proba(X_test_scaled_df)[:, 1]

# Print classification report for test set
print("Logistic Regression Test Set Classification Report")
print(classification_report(y_test, y_test_pred_lr))

# Print AUC score for test set
print(f"AUC (Test): {roc_auc_score(y_test, y_test_pred_proba_lr):.4f}")

In [ ]:
# Calculate ROC curve for test set
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba_lr)

# Calculate AUC score
auc_score = roc_auc_score(y_test, y_test_pred_proba_lr)

# Get the coefficients of the model
coefficients = np.append(best_lr_model.intercept_, best_lr_model.coef_.flatten())
logistic_df = pd.DataFrame(data=coefficients, index=['Intercept'] + [col for col in X_train_scaled_df.columns], columns=['Value']).sort_values('Value', ascending=False)

# Create a 1x2 subplot layout
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot ROC curve
axs[0].plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {auc_score:.4f})')
axs[0].plot([0, 1], [0, 1], color='gray', linestyle='--')  # Plotting the diagonal line for random model
axs[0].set_xlim([0.0, 1.0])
axs[0].set_ylim([0.0, 1.05])
axs[0].set_xlabel('False Positive Rate (FPR)')
axs[0].set_ylabel('True Positive Rate (TPR)')
axs[0].set_title('Receiver Operating Characteristic (ROC) Curve')
axs[0].legend(loc='lower right')

# Plot logistic regression coefficients
axs[1].barh(logistic_df.index, logistic_df['Value'], color='salmon')
axs[1].set_xlabel('Coefficient Value')
axs[1].set_ylabel('Feature Name')
axs[1].set_title('Logistic Regression Coefficients')

plt.tight_layout()
plt.show()

In [ ]:
# Function to plot impact of top features on default probability
def plot_feature_impact(top_features, model_name):
    plt.figure(figsize=(15, 3))  # Adjust figsize as needed
    num_features = len(top_features)
    num_cols = 3  # Number of columns for subplots

    # Calculate number of rows needed
    num_rows = (num_features // num_cols) + (num_features % num_cols > 0)

    for i, feature in enumerate(top_features):
        plt.subplot(num_rows, num_cols, i + 1)
        sns.regplot(x=X[feature], y=y, logistic=True, ci=None, scatter_kws={"s": 10, "alpha": 0.5})
        plt.title(f'Impact of {feature} on Default ({model_name})')
        plt.xlabel(feature)
        plt.ylabel('Probability of Default')
        plt.tight_layout()

    plt.tight_layout()
    plt.show()

# Example usage for Gradient Boosting (GB) model
top_features_lr = logistic_df.head(3).index  # Select top 3 features for Gradient Boosting
plot_feature_impact(top_features_lr, 'Logistic Regression')

In [ ]:
# Compute confusion matrix for test set
cm = confusion_matrix(y_test, y_test_pred_lr)

# Create ConfusionMatrixDisplay object
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Non-Default', 'Default'])

# Plot confusion matrix with annotations
fig, ax = plt.subplots(figsize=(6,4))
disp.plot(cmap='Blues', ax=ax, values_format='.4g')

plt.title('Confusion Matrix - Logistic Regression (Test Set)')
plt.tight_layout()
plt.show()

# Decision Tree Model
### It is a supervised machine learning algorithm used for both classification and regression tasks. It is a flowchart-like structure where each internal node represents a "decision" on a feature (or attribute), each branch represents the outcome of that decision, and each leaf node represents the final decision or the prediction label (class or value).
 - **Binary Splitting**: Decision Trees use recursive binary splitting to partition the data into subsets based on the values of features that best separate the target variable.
 - **Non-linear Model**: They can capture non-linear relationships between features and the target variable, making them suitable for complex datasets.
 - **Feature Importance**: They can rank features by their importance in predicting the target variable, based on metrics such as information gain (for classification) or variance reduction (for regression).
 - **Handling Mixed Data Types**: Decision Trees can handle both numerical and categorical data without requiring feature scaling.
 - **Splitting Criteria**: Common criteria for splitting nodes include Gini impurity and entropy for classification tasks, and variance reduction or mean squared error for regression tasks.

### CONS
- **Overfitting**: Without proper constraints (like pruning or limiting tree depth), Decision Trees can overfit the training data, capturing noise and details that do not generalize well to unseen data.

#### Validation Set

In [ ]:
%%time
param_grid_dt = {
    'max_depth': [None, 10, 20, 1, 2], #Maximum depth of the tree. It controls the maximum number of levels in the decision tree
                            #None: The tree is expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
                            #20: Limits the maximum depth to 30 levels.
    
    'min_samples_split': [2, 3, 10], #This param. ensures that a node in the decision tree must have at least 2 samples to be 
                                    #considered for further splitting.
                                    #It helps control the complexity of the tree and can prevent the model from learning noise.
    
    'min_samples_leaf': [1,2,5,10] #This parameter sets the minimum number of samples required to be at a leaf node.
                            #It ensures that each leaf node contains a sufficient number of samples to make robust predictions and prevents the model 
                            #from being too specific to the training data.
    }                        
# Initialize the DecisionTreeClassifier
dt_model = DecisionTreeClassifier(random_state=42)

# Initialize the GridSearchCV with the DecisionTreeClassifier and param_grid_dt
grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid_dt, cv=10, n_jobs=-1, verbose=1)

# Fit the GridSearchCV to the training data
grid_search.fit(X_train_scaled_df, y_train_sm)

# Get the best parameters from the grid search
best_params_dt = grid_search.best_params_
print(f"Best parameters found: {best_params_dt}")

# Get the best model from the grid search
best_dt_model = grid_search.best_estimator_

# Validate the model on the validation set
y_val_pred_dt = best_dt_model.predict(X_val_scaled_df)
y_val_pred_proba_dt = best_dt_model.predict_proba(X_val_scaled_df)[:, 1]

# Print classification report for validation set
print("Decision Tree Validation Set Classification Report")
print(classification_report(y_val, y_val_pred_dt))

# Print AUC score for validation set
print(f"AUC (Validation): {roc_auc_score(y_val, y_val_pred_proba_dt):.4f}")

#### Test Set

In [ ]:
%%time
#Initialize the DecisionTreeClassifier
dt_model = DecisionTreeClassifier(random_state=42,**best_params_dt)

# Fit the GridSearchCV to the training data
dt_model.fit(X_train_scaled_df, y_train_sm)

# Validate the model on the validation set
y_test_pred_dt = dt_model.predict(X_test_scaled_df)
y_test_pred_proba_dt = dt_model.predict_proba(X_test_scaled_df)[:, 1]

# Print classification report for validation set
print("Decision Tree Validation Set Classification Report")
print(classification_report(y_test, y_test_pred_dt))

# Print AUC score for validation set
print(f"AUC (Validation): {roc_auc_score(y_test, y_test_pred_proba_dt):.4f}")

In [ ]:
# Calculate ROC curve for test set
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba_dt)

# Calculate AUC score
auc_score = roc_auc_score(y_test, y_test_pred_proba_dt)

# Feature importances for Decision Tree
dt_importances = best_dt_model.feature_importances_
dt_df = pd.DataFrame(data=dt_importances, index=X_train_scaled_df.columns, columns=['Value']).sort_values('Value', ascending=False)

# Create a 1x2 subplot layout
fig, axs = plt.subplots(1, 2, figsize=(16, 5))

# Plot ROC curve
axs[0].plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {auc_score:.4f})')
axs[0].plot([0, 1], [0, 1], color='gray', linestyle='--')  # Plotting the diagonal line for random model
axs[0].set_xlim([0.0, 1.0])
axs[0].set_ylim([0.0, 1.05])
axs[0].set_xlabel('False Positive Rate (FPR)')
axs[0].set_ylabel('True Positive Rate (TPR)')
axs[0].set_title('Receiver Operating Characteristic (ROC) Curve')
axs[0].legend(loc='lower right')

# Plot Decision Tree feature importances
axs[1].barh(dt_df.index, dt_df['Value'], color='green')
axs[1].set_xlabel('Feature Importance')
axs[1].set_ylabel('Feature Name')
axs[1].set_title('Decision Tree Feature Importances')

plt.tight_layout()
plt.show()

In [ ]:
# Function to plot impact of top features on default probability
def plot_feature_impact(top_features, model_name):
    plt.figure(figsize=(15, 3))  # Adjust figsize as needed
    num_features = len(top_features)
    num_cols = 3  # Number of columns for subplots

    # Calculate number of rows needed
    num_rows = (num_features // num_cols) + (num_features % num_cols > 0)

    for i, feature in enumerate(top_features):
        plt.subplot(num_rows, num_cols, i + 1)
        sns.regplot(x=X[feature], y=y, logistic=True, ci=None, scatter_kws={"s": 10, "alpha": 0.5})
        plt.title(f'Impact of {feature} on Default ({model_name})')
        plt.xlabel(feature)
        plt.ylabel('Probability of Default')
        plt.tight_layout()

    plt.tight_layout()
    plt.show()

# Example usage for Gradient Boosting (GB) model
top_features_dt = dt_df.head(3).index  # Select top 3 features for Gradient Boosting
plot_feature_impact(top_features_dt, 'Decision Tree')

In [ ]:
# Compute confusion matrix for test set
cm = confusion_matrix(y_test, y_test_pred_dt)

# Create ConfusionMatrixDisplay object
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Non-Default', 'Default'])

# Plot confusion matrix with annotations
fig, ax = plt.subplots(figsize=(6,4))
disp.plot(cmap='Blues', ax=ax, values_format='.4g')

plt.title('Confusion Matrix - Decision Tree (Test Set)')
plt.tight_layout()
plt.show()

_______________________
# AdaBoost Model
#### It is an ensemble learning algorithm that combines multiple weak learners, typically decision stumps, to create a strong predictive model by iteratively focusing on and correcting the errors of previous learners.
- **Boosting Performance**: normally gives a high accuracy by combining the strenghts of multiple weak learners and it reduces variance and bias
- **Robust against overfitting**: By focusing on the hardest-to-classify samples, AdaBoost inherently reduces the risk of overfitting, especially when used with simple weak learners.
- **Handling Imbalanced Data**: It assigns higher weights to misclassified instances, which helps in handling imbalanced datasets where certain classes are underrepresented.

### CONS
- **Sensitive to Noisy Data**:can be very sensitive to noisy data and outliers, as it will assign higher weights to these misclassified instances, potentially degrading model performance.
- **Computational Complexity** : The iterative nature of AdaBoost, where each weak learner is trained sequentially, can lead to longer training times compared to other algorithms, especially with large datasets.

#### Validation Set

In [ ]:
%%time
param_grid_ada_val = {'n_estimators': [530, 520, 536, 538],'learning_rate': [1.72, 1.73, 1.69,1.68]}

# Initialize AdaBoost model
ada_model = AdaBoostClassifier(algorithm='SAMME.R',random_state=42)

# Set up the grid search with cross-validation
grid_search_ada_val = GridSearchCV(estimator=ada_model, param_grid=param_grid_ada_val, scoring='roc_auc', cv=10, n_jobs=-1, verbose=2, error_score='raise')

# Fit the GridSearchCV to the training data
grid_search_ada_val.fit(X_train_scaled_df, y_train_sm)

# Get the best parameters from the grid search
best_params_ada_val = grid_search_ada_val.best_params_
print(f"Best parameters found: {best_params_ada_val}")

# Get the best estimator
best_ada_model_val = grid_search_ada_val.best_estimator_

# Validate the model on the validation set
y_val_pred_ada = best_ada_model_val.predict(X_val_scaled_df)
y_val_pred_proba_ada = best_ada_model_val.predict_proba(X_val_scaled_df)[:, 1]

# Print classification report for validation set
print("Validation Set Classification Report")
print(classification_report(y_val, y_val_pred_ada))

# Print AUC score for validation set
print(f"AUC (Validation): {roc_auc_score(y_val, y_val_pred_proba_ada):.4f}")

#### Test Set

In [ ]:
%%time
ada_model = AdaBoostClassifier(algorithm='SAMME.R',random_state=42)
# Fit the GridSearchCV to the training data
ada_model.fit(X_train_scaled_df, y_train_sm)

# Evaluate the final model on the test data
y_test_pred_ada =ada_model.predict(X_test_scaled_df)
y_test_pred_proba_ada =ada_model.predict_proba(X_test_scaled_df)[:, 1]

# Print classification report for test set
print("Test Set Classification Report")
print(classification_report(y_test, y_test_pred_ada))

# Print AUC score for test set
print(f"AUC (Test): {roc_auc_score(y_test, y_test_pred_proba_ada):.4f}")

In [ ]:
# Calculate ROC curve for AdaBoost
fpr_ada, tpr_ada, thresholds_ada = roc_curve(y_test, y_test_pred_proba_ada)

# Plotting ROC curve and confusion matrix
plt.figure(figsize=(14, 6))

# Plotting ROC curve for AdaBoost
plt.subplot(1, 2, 1)
plt.plot(fpr_ada, tpr_ada, color='blue', lw=2, label='ROC curve (AdaBoost)')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Plotting the diagonal line for random model
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve - AdaBoost')
plt.legend(loc='lower right')

# Calculate and print AUC score for AdaBoost
auc_score_ada = roc_auc_score(y_test, y_test_pred_proba_ada)
plt.text(0.7, 0.3, f'AUC Score: {auc_score_ada:.4f}', ha='center', va='center', transform=plt.gca().transAxes)

# Compute confusion matrix for AdaBoost
cm_ada = confusion_matrix(y_test, y_test_pred_ada)

# Create ConfusionMatrixDisplay object for AdaBoost
disp_ada = ConfusionMatrixDisplay(confusion_matrix=cm_ada, display_labels=['Non-Default', 'Default'])

# Plot confusion matrix with annotations for AdaBoost
plt.subplot(1, 2, 2)
disp_ada.plot(cmap='Blues', ax=plt.gca(), values_format='.4g')
plt.title('Confusion Matrix - AdaBoost (Test Set)')
plt.tight_layout()

plt.show()

_______________________
# Gradient Boosting Model
#### It is used for regression and classification tasks. It builds an ensemble of decision trees in a stage-wise fashion, where each subsequent tree attempts to correct the errors of the previous trees.
- **High Predictive Accuracy**: delivers superior predictive performance compared to single decision trees or less complex ensemble methods.
- **Flexibility**: Capable of handling a variety of loss functions (e.g., regression, classification, ranking).
- **Feature Importance**: Provides insights into the importance of features in the dataset, aiding in feature selection and understanding model behavior.
- **Handles Various Data Types**: Works with both numerical and categorical data.

### CONS
- **Computationally Intensive**:Training gradient boosting models can be time-consuming and resource-intensive, especially with large datasets.
- **Prone to Overfitting**: Without proper tuning, gradient boosting models can overfit to the training data, leading to poor generalization on unseen data.
- **Sensitivity to Noisy Data**: Gradient Boosting can be sensitive to noisy data and outliers, which might necessitate additional data preprocessing steps.

#### Validation Set

In [ ]:
%%time
# Define parameter grid for Gradient Boosting
param_grid_gb_val = {
    'n_estimators': [200, 210], # Refers to the number of boosting stages (or trees) that will be iteratively built.
                                    # Each boosting stage improves the model by correcting errors of the previous stages.
                                    # After a certain amount of estimators, there's good chances of overfitting.
                                    # higher values makes the model more complex but also we risk overfitting 
    
    'learning_rate': [0.25, 0.3], #controls the contribution of each tree to the model. It scales the contribution 
                                    # of each tree by multiplying the predictions of each tree by this learning rate.
                                    # lower learning rate makes the model more robust by shrinking the contribution of each tree. This reduces
                                    # overfitting.
    
    'max_depth': [10, 11]       # Determines the maximum depth of each tree in the boosting process. Depth is the number of nodes from 
                                    # the root to the farthest leaf node.
                                    # Helps prevent overfitting. A deeper tree can model more complex relationships in the data but may also 
                                    # memorize noise in the training data, reducing generalization to new data.
}

# Initialize Gradient Boosting model
gb_model = GradientBoostingClassifier(random_state=42)

# Set up the GridSearchCV with the model and parameter grid
grid_search_val = GridSearchCV(estimator=gb_model, param_grid=param_grid_gb_val, scoring='roc_auc', cv=10, n_jobs=-1)

# Fit the GridSearchCV to the training data
grid_search_val.fit(X_train_scaled_df, y_train_sm)

# Get the best parameters from the grid search
best_params_gb = grid_search_val.best_params_
print(f"Best parameters found: {best_params_gb}")

# Get the best model from the grid search
best_gb_model_val = grid_search_val.best_estimator_

# Make predictions on the test data
y_val_pred_gb_val = best_gb_model_val.predict(X_val_scaled_df)
y_val_pred_proba_gb_val = best_gb_model_val.predict_proba(X_val_scaled_df)[:, 1]

# Print classification report for validation set
print("Validation Set Classification Report")
print(classification_report(y_val, y_val_pred_gb_val))

# Print AUC score for validation set
print(f"AUC (Validation): {roc_auc_score(y_val, y_val_pred_proba_gb_val):.4f}")

#### Test Set

In [ ]:
# Initialize Gradient Boosting model
gb_model = GradientBoostingClassifier(random_state=42)

# Fit the GridSearchCV to the training data
gb_model.fit(X_train_scaled_df, y_train_sm)

# Evaluate the final model on the test data
y_test_pred_gb_test = gb_model.predict(X_test_scaled_df)
y_test_pred_proba_gb_test = gb_model.predict_proba(X_test_scaled_df)[:, 1]

# Print classification report for test set
print("Test Set Classification Report")
print(classification_report(y_test, y_test_pred_gb_test))

# Print AUC score for test set
print(f"AUC (Test): {roc_auc_score(y_test, y_test_pred_proba_gb_test):.4f}")

In [ ]:
# Calculate ROC curve for test set
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba_gb_test)

# Calculate AUC score
auc_score = roc_auc_score(y_test, y_test_pred_proba_gb_test)

# Feature importances for Gradient Boosting
gb_importances = gb_model.feature_importances_
gb_df = pd.DataFrame(data=gb_importances, index=X_train.columns, columns=['Value']).sort_values('Value', ascending=False)

# Create a 1x2 subplot layout
fig, axs = plt.subplots(1, 2, figsize=(16, 5))
# Plot ROC curve
axs[0].plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {auc_score:.4f})')
axs[0].plot([0, 1], [0, 1], color='gray', linestyle='--')  # Plotting the diagonal line for random model
axs[0].set_xlim([0.0, 1.0])
axs[0].set_ylim([0.0, 1.05])
axs[0].set_xlabel('False Positive Rate (FPR)')
axs[0].set_ylabel('True Positive Rate (TPR)')
axs[0].set_title('Receiver Operating Characteristic (ROC) Curve')
axs[0].legend(loc='lower right')

# Plot Gradient Boosting feature importances
axs[1].barh(gb_df.index, gb_df['Value'], color='green')
axs[1].set_xlabel('Feature Importance')
axs[1].set_ylabel('Feature Name')
axs[1].set_title('Gradient Boosting Feature Importances')

plt.tight_layout()
plt.show()

In [ ]:
# Function to plot impact of top features on default probability
def plot_feature_impact(top_features, model_name):
    plt.figure(figsize=(15, 3))  # Adjust figsize as needed
    num_features = len(top_features)
    num_cols = 3  # Number of columns for subplots

    # Calculate number of rows needed
    num_rows = (num_features // num_cols) + (num_features % num_cols > 0)

    for i, feature in enumerate(top_features):
        plt.subplot(num_rows, num_cols, i + 1)
        sns.regplot(x=X[feature], y=y, logistic=True, ci=None, scatter_kws={"s": 10, "alpha": 0.5})
        plt.title(f'Impact of {feature} on Default ({model_name})')
        plt.xlabel(feature)
        plt.ylabel('Probability of Default')
        plt.tight_layout()

    plt.tight_layout()
    plt.show()

# Example usage for Gradient Boosting (GB) model
top_features_gb = gb_df.head(3).index  # Select top 3 features for Gradient Boosting
plot_feature_impact(top_features_gb, 'Gradient Boosting')

In [ ]:
# Compute confusion matrix for test set
cm = confusion_matrix(y_test, y_test_pred_gb_test)

# Create ConfusionMatrixDisplay object
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Non-Default', 'Default'])

# Plot confusion matrix with annotations
fig, ax = plt.subplots(figsize=(6, 4))
disp.plot(cmap='Blues', ax=ax, values_format='.4g')

plt.title('Confusion Matrix - Gradient Boosting Classifier (Test Set)')
plt.tight_layout()
plt.show()

__________________

### Some analysis regarding the output
#### Following, we can see the dataset TEST with:
- First column is the id of the individual
- Second column is the actual default/non default
- Each sequent column is the predicted default/non default of each model

In [ ]:
y_test_actual = y_test  # Actual default/non-default values from the test set
predictions={
    'Actual': y_test_actual,
    'XGBoost': y_test_pred_xgb,
    'RandomForest': y_test_pred_rf,
    'LogisticRegression': y_test_pred_lr,
    'Decision Tree': y_test_pred_dt,
    'AdaBoost': y_test_pred_ada,
    'Gradient Boosting': y_test_pred_gb_test   
}

predictions_df = pd.DataFrame(predictions)
from IPython.display import display
display(predictions_df)

In [ ]:
# Filter the DataFrame for defaulted observations (Actual == 1)
defaulted_df = predictions_df[predictions_df['Actual'] == 1]

# Get model names
model_names = ['XGBoost', 'RandomForest', 'LogisticRegression', 'Decision Tree', 'AdaBoost', 'Gradient Boosting']

# Initialize a DataFrame to hold the number of correct predictions for model pairs
pair_matrix = pd.DataFrame(index=model_names, columns=model_names, data=0)

# Calculate the number of correct predictions for each pair of models
for model1, model2 in combinations(model_names, 2):
    # Get predictions for the pair of models
    pred1 = defaulted_df[model1]
    pred2 = defaulted_df[model2]
    
    # Calculate the number of correct predictions where both models are correct
    correct_predictions = np.sum((pred1 == defaulted_df['Actual']) & (pred2 == defaulted_df['Actual']))
    
    # Store the number of correct predictions in the matrix
    pair_matrix.loc[model1, model2] = correct_predictions
    pair_matrix.loc[model2, model1] = correct_predictions

# Display the matrix
print(pair_matrix)

In [ ]:
# Filter the DataFrame for defaulted observations (Actual == 1)
defaulted_df = predictions_df[predictions_df['Actual'] == 1]

# Get model names
model_names = ['XGBoost', 'RandomForest', 'LogisticRegression', 'Decision Tree', 'AdaBoost', 'Gradient Boosting']

# Initialize a DataFrame to hold the number of correct predictions for model pairs
pair_matrix = pd.DataFrame(index=model_names, columns=model_names, data=0)

# Calculate the number of correct predictions for each pair of models
for model1, model2 in combinations(model_names, 2):
    # Get predictions for the pair of models
    pred1 = defaulted_df[model1]
    pred2 = defaulted_df[model2]
    
    # Calculate the number of correct predictions where both models are correct
    correct_predictions = np.sum((pred1 == defaulted_df['Actual']) & (pred2 == defaulted_df['Actual']))
    
    # Store the number of correct predictions in the matrix
    pair_matrix.loc[model1, model2] = correct_predictions
    pair_matrix.loc[model2, model1] = correct_predictions

fig, ax = plt.subplots(figsize=(10, 8))
cax = ax.matshow(pair_matrix, cmap='viridis')

# Add color bar
fig.colorbar(cax)

# Set axis labels
ax.set_xticks(np.arange(len(model_names)))
ax.set_yticks(np.arange(len(model_names)))
ax.set_xticklabels(model_names)
ax.set_yticklabels(model_names)

# Rotate the x labels
plt.xticks(rotation=90)

# Annotate each cell with the numeric value
for i in range(len(model_names)):
    for j in range(len(model_names)):
        ax.text(j, i, pair_matrix.iloc[i, j], va='center', ha='center')

plt.title('Number of Correct Predictions for Each Pair of Models')
plt.show()

In [ ]:
# Function to add counts on bars
def add_bar_labels(ax, bars):
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, yval, int(yval), va='bottom', ha='center')

# Plot for REASON distribution
total_reason = data['REASON'].value_counts().sort_index()
default_reason = data[data['BAD'] == 1]['REASON'].value_counts().sort_index()

fig, ax = plt.subplots(figsize=(8, 5))
total_bars = ax.bar(total_reason.index - 0.2, total_reason, width=0.4, label='Total', align='center')
default_bars = ax.bar(default_reason.index + 0.2, default_reason, width=0.4, label='Defaulters', align='center')

# Adding labels and title for REASON
ax.set_xticks([0, 1])
ax.set_xticklabels(['DebtCon', 'HomeImp'])  # Assuming your REASON labels are DebtCon = 0, HomeImp = 1
ax.set_ylabel('Count')
ax.set_title('Distribution of Total vs Defaulters for REASON')
ax.legend()

# Add counts on the bars for REASON
add_bar_labels(ax, total_bars)
add_bar_labels(ax, default_bars)

plt.tight_layout()
plt.show()

# Plot for JOB distribution
total_job = data['JOB'].value_counts().sort_index()
default_job = data[data['BAD'] == 1]['JOB'].value_counts().sort_index()

fig, ax = plt.subplots(figsize=(10, 6))
total_bars = ax.bar(total_job.index - 0.2, total_job, width=0.4, label='Total', align='center')
default_bars = ax.bar(default_job.index + 0.2, default_job, width=0.4, label='Defaulters', align='center')

# Adding labels and title for JOB
job_labels = ['Other', 'Office', 'Mgr', 'ProfExe', 'Sales', 'Self']  # JOB labels as per your mapping
ax.set_xticks(range(6))
ax.set_xticklabels(job_labels)
ax.set_ylabel('Count')
ax.set_title('Distribution of Total vs Defaulters for JOB')
ax.legend()

# Add counts on the bars for JOB
add_bar_labels(ax, total_bars)
add_bar_labels(ax, default_bars)

plt.tight_layout()
plt.show()